In [8]:
import os
import librosa
import soundfile as sf
import numpy as np
import pandas as pd
import random

In [9]:
def time_shift(signal, sr, shift_ms=500):
    shift_samples = int(shift_ms * sr / 1000)
    direction = random.choice([-1, 1])
    shifted_signal = np.roll(signal, direction * shift_samples)
    if direction == 1:
        shifted_signal[:shift_samples] = 0
    else:
        shifted_signal[shift_samples:] = 0

    shifted_signal=shifted_signal/np.max(np.abs(shifted_signal))
    return shifted_signal

def pitch_shift(signal, sr, n_steps=2):
    # Remove NaNs or infs if any
    signal = np.nan_to_num(signal)

    # Normalize safely
    max_val = np.max(np.abs(signal))
    if max_val > 0:
        signal = signal / max_val

    # Apply pitch shift
    shifted_signal = librosa.effects.pitch_shift(y=signal, sr=sr, n_steps=n_steps)

    # Final normalization
    max_val = np.max(np.abs(shifted_signal))
    if max_val > 0:
        shifted_signal = shifted_signal / max_val

    return shifted_signal


def white_noiser(signal,noise_factor=random.uniform(0.01, 0.1)): #0.3
    noise = np.random.normal(0,signal.std(),signal.size)
    augmented_signal = signal + noise * noise_factor
    augmented_signal=augmented_signal/np.max(np.abs(augmented_signal))
    return augmented_signal

def random_gain(signal,min_gain=0.15,max_gain=1.7):
    gain_factor = random.uniform(min_gain,max_gain)
    signal=signal*gain_factor
    final_signal = signal / np.max(np.abs(signal)) 
    return final_signal


def apply_time_stretch(signal, target_length=None):
    rate=random.uniform(0.85, 1.15)
    stretched_audio = librosa.effects.time_stretch(signal,rate=rate)

    if target_length is None:
            target_length = len(signal)

    if len(stretched_audio) > target_length:
        stretched_audio = stretched_audio[:target_length]
    else:
        stretched_audio = np.pad(stretched_audio, (0, target_length - len(stretched_audio)))
    
    stretched_audio = stretched_audio / np.max(np.abs(stretched_audio))
    return stretched_audio



##### AUGMENTATION TECHNIQUES
###### white_noiser(pitch_shift(A))
###### random_gain(apply_time_stretch(A))
###### pitch_shift(random_gain(A))
###### apply_time_stretch(pitch_shift(A))

In [10]:
dataset_path=r'D:\Projects\MoodMate\augmented_dataset'

for emotion in os.listdir(dataset_path):
    if emotion=="sad":
        for audio in os.listdir(os.path.join(dataset_path,emotion)):
            data_count=len(os.listdir(os.path.join(dataset_path,emotion))) +1
            signal,sr= librosa.load(os.path.join(dataset_path,emotion,audio))
            
            augmented_signal_1=white_noiser(pitch_shift(signal,sr))
            sf.write(os.path.join(dataset_path,emotion,emotion) + "_"+ str(data_count) + '.wav',augmented_signal_1,sr)
            data_count+=1

            augmented_signal_2=random_gain(pitch_shift(signal,sr))
            sf.write(os.path.join(dataset_path,emotion,emotion) + "_"+ str(data_count) + '.wav',augmented_signal_2,sr)
            data_count+=1

            augmented_signal_3=pitch_shift(random_gain(signal),sr)
            sf.write(os.path.join(dataset_path,emotion,emotion) + "_"+ str(data_count) + '.wav',augmented_signal_3,sr)
            data_count+=1

            augmented_signal_4=apply_time_stretch(pitch_shift(signal,sr))
            sf.write(os.path.join(dataset_path,emotion,emotion) + "_"+ str(data_count) + '.wav',augmented_signal_4,sr)
            data_count+=1
        
print("DATA AUGMENTATION COMPLETED")

        

C:\Users\sudha\AppData\Local\Temp\ipykernel_8444\883601570.py:36: RuntimeWarning: invalid value encountered in divide
  augmented_signal=augmented_signal/np.max(np.abs(augmented_signal))
C:\Users\sudha\AppData\Local\Temp\ipykernel_8444\883601570.py:42: RuntimeWarning: invalid value encountered in divide
  final_signal = signal / np.max(np.abs(signal))
C:\Users\sudha\AppData\Local\Temp\ipykernel_8444\883601570.py:58: RuntimeWarning: invalid value encountered in divide
  stretched_audio = stretched_audio / np.max(np.abs(stretched_audio))


DATA AUGMENTATION COMPLETED
